# Emlékeztető

Az elmúlt hetekben 
- Objektum orientált programozás: osztály definíció (`class`), dunder metódusok stb. 
- `numpy` és
- `matplotlib` könyvtár
volt terítéken. A `matplotlib` könytárat csak példákon keresztül néztük.

A mai alkalommal 

- visszatérünk egy rövid példa erejéig az öröklödésre 

- példákon keresztül megnézzük a `pandas` könyvtár használatát.

- ha marad idő megnézzük a Newton módszer alkalmazását.

# Kis kitérő: Öröklődés

Korábban szerepelt ehhez hasonló példa. Legyenek `Polygon`, `Rectangle`, `Square` osztályaink.
Minden négyzet téglalap és minden téglalap sokszög. Egy sokszöget a csúcsok felsorolásával adhatunk meg.
Ebből a kerület, terület kiszámolható és eldönthető, hogy a sokszög konvex-e. Ha akarjuk pl. a `Matplotlib` könyvtárral ki is rajzolhatjuk a sokszöget.

Menetközben síkvektorokkal akarunk számolni. Erre kényelmes a `numpy` könyvtárat használni.

In [ ]:
import numpy as np
from itertools import pairwise
import matplotlib.pyplot as plt

In [ ]:
class Point(np.ndarray):
    def __new__(cls, point):
        point = np.asarray(point)
        assert point.shape[-1] == 2
        return point.view(cls)

    def __repr__(self):
        if self.ndim > 1:
            return f"Array of Points"
        else:
            a, b =  self
        return f"Point{(a,b)}"
    
def rotate90(x):
    return np.flip(x, axis=-1)*np.array([1, -1])

In [ ]:
Point((1,2))-Point((3,4))

In [ ]:
class Polygon:
    def __init__(self, nodes):
        self.nodes = tuple((Point(node) for node in nodes))

    def area(self):
        nodes = np.array(self.nodes)
        area = sum(v1[0]*v2[1]-v1[1]*v2[0] for v1, v2 in pairwise(nodes-nodes[0]))
        return abs(area)/2

    def edges(self):
        nodes = np.array(self.nodes)
        return nodes-np.roll(nodes, 1, axis=0)
    
    def perimeter(self):
        return sum(np.linalg.norm(edge) for edge in self.edges())
    
    def is_convex(self):
        edges = self.edges()
        sgn = np.einsum('ij,ij->i', rotate90(edges), np.roll(edges, 1, axis=0))
        return (min(sgn)*max(sgn)>=0)

    def draw(self,  *args, fill=False, **kwargs):
        nodes = np.array(self.nodes + self.nodes[:1])
        if fill: plt.fill(*nodes.T, alpha=0.5)
        plt.plot(*nodes.T, *args, **kwargs)
        plt.axis('equal')

    def __repr__(self):
        return f'{type(self).__name__}({[tuple(x) for x in self.nodes]})'

In [ ]:
class Rectangle(Polygon):

    def __init__(self, P1, P2, r):
        """
        P1, P2 froms an edge and r is the ratio of edge lengths.

        Assuming that e is P1P2 rotated counterclockwise by 90 degree
        the vertices are:
        P1, P2, P2+r*e, P1+r*e
        """
        P1, P2 = Point(P1), Point(P2)
        v = rotate90(P2-P1)*r
        super(Rectangle, self).__init__([P1, P2, P2+v, P1+v])

    def is_convex(self):
        return True

In [ ]:
class Square(Rectangle):

    def __init__(self, P1, P2):
        super(Square, self).__init__(P1, P2, 1)

In [ ]:
rotate90(np.array([[0, 1], [1, 0]]))

In [ ]:
points = np.array([[0,0], [1,1]])
s = Square(points[0], points[1])
t = Rectangle(points[0], points[1], 2)
p = Polygon(points)
for x in [s, t, p]:
    print(f"{x} kerület: {x.perimeter():.2f}, konvex: {x.is_convex()}, terület: {x.area():.2f}")


Vegyük észre, hogy az `__repr__` metódust csak egyszer implementáltuk, mégis minden egyednek a saját típusát írja ki!
Az előző kód blokkot futtassuk le azután is, hogy a `draw` metódust implementáltuk.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
points = np.array([[0,0], [2,0], [2,2], [1,1], [1,2], [0,2], [0.5,1]])
p = Polygon(list(points))
print(f"{p} konvex?: {p.is_convex()}, {p.area()=}")
p.draw('o-', fill=True)
plt.grid(); plt.show()


In [ ]:
plt.plot(*np.array(s.nodes)[[0,1,2,3,0]].T)
plt.grid()
plt.axis('equal')
# plt.axis('off')


In [ ]:
t.draw('o-', alpha=0.5)
s.draw('o:', markersize=10, alpha=0.5, fill=True)
p.draw('o-', mfc="yellow", mec='r', alpha=0.5)
plt.show()

In [ ]:
p = Square([0,0], [1,0])
p.draw(fill=True)

# Táblázatos adatok kezelése (pandas - tabular datasets in Python)

<table border="1" class="dataframe">
    <thead>
        <tr style="text-align: right;">
            <th></th>
            <th>name</th>
            <th>age</th>
            <th>average</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>0</th>
            <td>Anna</td>
            <td>20</td>
            <td>4.5</td>
        </tr>
        <tr>
            <th>1</th>
            <td>Bob</td>
            <td>19</td>
            <td>3.9</td>
        </tr>
        <tr>
            <th>2</th>
            <td>Cecil</td>
            <td>22</td>
            <td>4.1</td>
        </tr>
    </tbody>
</table>

A `pandas` könyvtár az alábbi feladatokra kínál megoldást:
* táblázatos adatok kezelése
* műveletek, csoportok képzése, statisztikák számolása
* idősoros adatok kezelése
* hiányzó adatok kezelése
* adatok összefésülése SQL-szerű műveletekkel
* sokféle input és output formátum támogatása

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
small_df = pd.DataFrame(
    [["Anna", 20, 4.5], ["Bob", 19, 3.9], ["Cecil", 22, 4.1]],
    columns=["name", "age", "average"]
)

small_df

In [ ]:
# rekordok listája, index értékekkel

df = pd.DataFrame(
    [["Anna", 20, 4.5], ["Bob", 19, 3.9], ["Cecil", 22, 4.1]],
    columns=["name", "age", "average"],
    index=["one", "two", "three"]
)

df

In [ ]:
df["name"] # pandas Series objektum

In [ ]:
df.loc[["one", "two"]]

In [ ]:
df.loc[["one", "two"], ["name", "age"]]

In [ ]:
df[["name", "age"]] # nincs loc, vagy df.loc[:, ["name", "age"]]

In [ ]:
df[df["average"] >= 4.0] # bool index, vagy df.loc[df["average"] >= 4]

Nagyon sok adat táblázatos formátumú. 

Statisztikában, klasszikus gépi tanulási algoritmusoknál a táblázat sorai egy-egy adatpontot reprezentálnak. 

A sor elemei a megfigyelt tulajdonságok, ezek nem feltétlenül azonos típusúak.

Egy példa a `Kaggle`-ről: **Időjárás Szegeden**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("budincsevity/szeged-weather")

print("Path to dataset files:", path)

In [ ]:
from pathlib import Path

path = Path(path)
print(*path.glob("*"), sep="\n")
csv_file = next(iter(path.glob("*.csv")))
print(csv_file)

In [ ]:
!head -n 5 $csv_file

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv(csv_file, nrows=5)
df.head()

In [ ]:
df = pd.read_csv(csv_file)
df.shape

## Információk az adat keret (`pandas.DataFrame`) oszlopairól

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

Látszik, hogy a  `Loud Cover` oszlop csupa nulla. Eldobhatjuk.

In [ ]:
df.drop(columns=["Loud Cover"], inplace=True)
df.columns

In [ ]:
df.isna().sum()

Honnan jönnek az `NA` értékek?

In [ ]:
df["date"] = pd.to_datetime(df["Formatted Date"], utc=True)
df.sort_values("date", inplace=True)

plt.plot(df.date, df.isna().any(axis=1).cumsum())
plt.grid()

In [ ]:
df["Precip Type"].value_counts()

In [ ]:
df.plot(x="Temperature (C)", y="Apparent Temperature (C)", kind="scatter", s=1)
plt.show()

In [ ]:
df["temp_diff"] = df["Apparent Temperature (C)"] - df["Temperature (C)"]

In [ ]:
sns.scatterplot(data=df.sample(1000), 
                x="Wind Speed (km/h)", y="temp_diff", size="Humidity", hue="Temperature (C)")
plt.show()

In [ ]:
sns.pairplot(df.sample(1000), diag_kind="kde"); plt.show()

In [ ]:
df["rain_or_snow"] = df["Precip Type"].map({"rain": 0, "snow": 1})

In [ ]:
sns.heatmap(df.loc(1)[df.dtypes!=np.dtype('O')].corr(), 
            annot=True, fmt=".2f", vmin=-1, vmax=1, cmap="coolwarm")
plt.show()

## Idősorrá (time-series) alakítás

In [ ]:
ts = df.drop(columns=["Formatted Date"])
ts["date"] = (
    pd.to_datetime(df["Formatted Date"], utc=True)
    .dt.tz_convert("Europe/Budapest")
)
ts.set_index("date", inplace=True)
ts.sort_index(inplace=True)
ts.head(3)

## Megjegyzés.

Már a beolvasáskor is a kívánt alakúra hozhatnánk az adat keretünket, de ez lassabb lenne.

In [ ]:
%%time
df0 = pd.read_csv(
    csv_file, 
    # index_col="Formatted Date", 
    parse_dates=["Formatted Date"],
#     converters={"Formatted Date": lambda x: pd.to_datetime(x, utc=True, format='ISO8601')},
)

In [ ]:
%%time
df1 = pd.read_csv(csv_file)
df1["Formatted Date"] = pd.to_datetime(df["Formatted Date"], utc=True)

In [ ]:
#ellenőrzés
(df0["Formatted Date"] == df1["Formatted Date"]).all()

Egy `datetime` típusú oszlopból a dátum elemei kiolvashatóak a `dt.hour`, `dt.year` stb. tulajdonságok segítségével.

In [ ]:
ts[ts.index.hour==12].shape # Index exetén a `dt` rész elhagyható.

In [ ]:
import seaborn as sns

plt.figure(figsize=(16, 4))
ax = sns.scatterplot(
    data=ts[ts.index.hour==12], 
    x="date", y="Temperature (C)", s=4
)
ax.set_title("Hőmérséklet Szegeden 12 órakor")
ax.set_xlabel("Év"); ax.grid(); plt.show()

In [ ]:
ax = ts.groupby(ts.index.hour)["Temperature (C)"].mean().plot(kind="bar", figsize=(8, 3))
ax.set_xlabel("óra")
ax.set_ylabel("átlag hőmérséklet (C)")
plt.show()

In [ ]:
ts["hónap"] = ts.index.month_name(locale="hu_HU.utf8")
ts["óra"]  = ts.index.hour
ts["nap"] = ts.index.day_of_year
ts["év"] = ts.index.year
ts.head()

In [ ]:
import seaborn as sns

plt.figure(figsize=(16, 4))
ax = sns.lineplot(
    data=ts[(ts.index.hour == 12)], 
    x="nap", 
    y="Temperature (C)", 
)
ax.set_title("Hőmérséklet Szegeden 12 órakor")
ax.grid(); plt.show()

In [ ]:
ts["Temperature (C)"].plot(figsize=(16, 4))
plt.show()

In [ ]:
ts["Temperature (C)"].iloc[:14*24].plot(figsize=(16, 4))
plt.grid(); plt.show()

In [ ]:
ts["Temperature (C)"].loc["2006-10-20":"2006-10-27"].plot(figsize=(16, 4))
plt.grid(); plt.show()

In [ ]:
ts_one_year = ts.loc["2011-03-01":"2012-02-28"]

In [ ]:
ts_one_year

In [ ]:
ts_one_year["Temperature (C)"].plot(figsize=(16, 4))
plt.show()

In [ ]:
ts_one_year['Temperature (C)'].resample('MS').apply("mean").plot(kind="barh")
plt.show()

In [ ]:
(ts['Temperature (C)']
    .resample("YS")
    .apply("mean")
    .plot(kind="barh")
)
plt.show()

In [ ]:
ts_one_year.resample('3MS')['Temperature (C)'].agg(['min','max', 'mean', 'count'])

**HF**: 
* Keressünk gyanús adapontokat. Nézzük meg a légnyomás oszlopot. Mi a mérési értékek eloszlása?
* Hány napon havazott?
* Mikor volt köd? Melyik a legködösebb hónap?

## `pandas.DataFrame` létrehozása, indexelés 

Néhány lehetőség:

In [ ]:
data = [{'a': 1, 'b':2}, {'a': 3, 'b':4}]
pd.DataFrame(data)

In [ ]:
pd.DataFrame({'a': [1, 3], 'b': [2, 4]})

In [ ]:
pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])

## Indexelés, `loc` és `iloc` segítségével

`loc` esetén index értékeket és oszlopneveket használunk

In [ ]:
ts.loc["2007-01-01":"2007-12-31", ["Summary", "Humidity", "Precip Type"]]

`iloc` esetén numerikus indexeket.

In [ ]:
df.iloc[:5, :5]

Egy data frame minden oszlopa egy `pandas.Series` objektum. Ezek indexelt numpy vectorok.

In [ ]:
col = ts["Summary"]
print(f"{type(col)=},\n{col.dtype=},\n{col.shape=},\n{col.ndim=},\n{col.name=}")
col.head()


In [ ]:
df[["Summary"]].head() ## Ez egy egy oszlopos DataFrame

In [ ]:
df.iloc[:5,:5].values ## Ez egy numpy tömb

In [ ]:
df["Temperature (C)"].values ## Ez egy numpy vector

### Kiírás fileba

In [ ]:
# help(pd.DataFrame.to_csv)

In [ ]:
df.iloc[:5, :5].to_csv(
    "/tmp/small.csv", # path
#     index=False,
#     sep=";",
#     decimal=",",
)
! cat /tmp/small.csv

### Más formátumba alakítás

In [ ]:
print(small_df.to_markdown(index=False))

In [ ]:
print(small_df.to_latex(float_format="{:.1f}".format))

In [ ]:
[f for f in dir(small_df) if f.startswith("to_")]

## Néhány `DataFrame`-ekkel végezhető művelet

- `numpy`-ban látott aggregáló műveletek (`min`, `max` `sum`, `mean`, `std`, `corr`)

- `groupby`: Csoportosítás adott oszlop, vagy oszlopok értékei szerint. 

- idősorokra `resample`

- `merge`: két `DataFrame`-ből egyet készít.

- `pivot_table`: Összefoglaló táblázat (EXCEL funkcióhoz hasonlít).

- Széles és hosszú formátumok közti konvertálás.

In [ ]:
df.loc(1)[df.dtypes == np.float64].corr()

In [ ]:
months = pd.date_range("2000-01-01", "2000-12-31", freq="ME").month_name(locale="hu_HU.utf8").values
months

In [ ]:
mean_temp = ts.groupby("hónap")[["Temperature (C)"]].mean()

mean_temp.columns = ["Havi átlaghőmérséklet"]
mean_temp = mean_temp.loc(0)[months]
mean_temp

In [ ]:
# több szempont szerint is lehet csoportosítani.
mean_temp = ts.groupby(["hónap", "Precip Type"])[["Temperature (C)"]].mean()

mean_temp.columns = ["Havi átlaghőmérséklet"]
mean_temp.loc(0)[months] 

In [ ]:
# Merge variációi: how lehet inner, left, right, outer
a = pd.DataFrame([[1, 2], [3, 4]], columns=["a", "b"])
b = pd.DataFrame([[1, 3], [5, 6]], columns=["a", "b"])

a.merge(b, on="a", 
        how="outer", # "inner", "left" or "right"
        suffixes=("_left", "_right")
       )

In [ ]:
# összefűzés
pd.concat((a,b), ignore_index=True)

In [ ]:
# Pivot_table
ts["óra/4"] = ts["óra"]//4
table = ts.pivot_table(index="hónap", columns="óra/4", values="Temperature (C)", aggfunc="mean")
table 

In [ ]:
table = table.loc(0)[months]
table.columns = [f"{i*4}-{(i+1)*4}" for i in range(6)]
table.columns.name = "Idősáv"
table

In [ ]:
# még egy példa
ts["Idősáv"] = pd.Categorical(
    (ts["óra"]//4).map(lambda i: f"{i*4}-{(i+1)*4}"), 
    categories=[f"{i*4}-{(i+1)*4}" for i in range(6)],
    ordered=True)
ts["Idősáv"]

In [ ]:
ts.pivot_table(
    index=["hónap", "Precip Type"], 
    columns="Idősáv", 
    values="Temperature (C)", 
    aggfunc="mean",
    observed=False
).loc(0)[months]

## A világ népessége

In [ ]:
import kagglehub
from pathlib import Path

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/world-population-dataset")

print("Path to dataset files:", path)
print(*Path(path).glob("*"), sep="\n")
csv_file = next(iter(Path(path).glob("*.csv")))

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.removesuffix(" Population")
df.columns = df.columns.str.removesuffix("/Territory")
df = df.rename(columns=lambda x: x.split(" (")[0])
df.columns 


In [ ]:
df.head()

In [ ]:
if "Capital" in df.columns:
    df.drop(columns=["CCA3", "Capital"], inplace=True) 

In [ ]:
df.head()

In [ ]:
df[df["Continent"]=="Europe"].head()

In [ ]:
df[df["Area"] <= 10]

In [ ]:
df[(df["Continent"]!="Europe") & (df["Area"] <= 100)]

In [ ]:
df.plot(x="Rank", y="World Population Percentage", kind="scatter", s=1, logy=True)
plt.show()

In [ ]:
sorted_by_rank_df = df.sort_values(by="Rank")

sorted_by_rank_df.head(n=10)

In [ ]:
sorted_by_rank_df[sorted_by_rank_df["Continent"]=="Europe"].head(n=20)

In [ ]:
europe_sorted_df = sorted_by_rank_df[sorted_by_rank_df["Continent"]=="Europe"]

k = 10

In [ ]:
# Ábra készítése plt-vel. Ez a legegyszerűbb, ha csak egyetlen ábránk lesz
k=16
plt.figure(figsize=(16, 6))
plt.bar(range(k), europe_sorted_df["2022"][:k])
plt.title("Countries in Europe with the largest population")
plt.xticks(range(k), labels=europe_sorted_df["Country"][:k], rotation=60, fontsize=14)
plt.show()

In [ ]:
# Ábra készítése az Axis objektummal.

fig, ax = plt.subplots(figsize=(16, 6))
ax.bar(range(k), europe_sorted_df["2022"][:k])
ax.set_title("Countries in Europe with the largest population")
ax.set_xticks(range(k), labels=europe_sorted_df["Country"][:k], rotation=45, fontsize=14)
ax.yaxis.set_major_formatter("{x:,.0f}")
plt.show()

In [ ]:
# ábra sns-el
ax = sns.barplot(data=europe_sorted_df.head(10), x="Country", y="2022")
ax.set_title("Countries in Europe with the largest population")
ax.xaxis.set_tick_params(rotation=45, labelsize=14)
ax.yaxis.set_major_formatter("{x:,.0f}")
ax.figure.set_size_inches(16, 6)
plt.show()

Kérdések:

* Melyik ország lakossága növekedett a legnagyobb mértékben 1970 és 2022 között?

* Melyik ország lakossága csökkent a legnagyobb mértékben 1970 és 2022 között?

* Földrészenként melyik a legnépesebb ország?

* A világ népességének mi a százalékos megoszlása földrészenként?

In [ ]:
ratio = (df["2022"] / df["1970"]).sort_values(ascending=False)

In [ ]:
ratio.head()

In [ ]:
country_name = df.loc[ratio.index[0], "Country"]

country_name

In [ ]:
df[df["Country"]==country_name]

In [ ]:
ratio.tail()

In [ ]:
country_name = df.loc[ratio.index[-1], "Country"]

country_name

In [ ]:
df[df["Country"]==country_name]

In [ ]:
grouped = df.groupby(["Continent"])

In [ ]:
df.loc[grouped["2022"].idxmax(), ["Continent", "Country", "2022"]]

In [ ]:
# percentage = grouped["World Population Percentage"].aggregate(sum)
percentage = grouped["World Population Percentage"].agg("sum")
percentage

In [ ]:
fig1, ax = plt.subplots(figsize=(10, 10))
ax.pie(
    percentage, 
    labels=percentage.index, 
    explode=(0, 0, 0.1, 0, 0, 0), 
    textprops={"fontsize": 14}, 
#     autopct="%1.1f%%"
    autopct=lambda p: f"{p:.1f}%"  if p>1 else ""
)
plt.show()

#### A földrészek népességének változása 1970-től 2022-ig

In [ ]:
grouped = df.groupby(["Continent"])
years = ["1970", "1980", "1990", "2000", "2010", "2015", "2020", "2022"]
df_contient_populations = grouped[years].sum()
df_contient_populations

### Széles formátum

In [ ]:
df_contient_populations

### Hosszú formátum

In [ ]:
df_long = df.melt(
    id_vars=["Continent", "Country"], #, "Rank", "World Population Percentage", "Area", "Growth Rate", "Density"], 
    value_vars=years, 
    var_name="Year", 
    value_name="Population"
)
df_long

- Szélesből hosszú formátumba  a `melt` metódussal,

- Hosszúból szélesbe a `pivot` metódussal

válthatunk

In [ ]:
df_wide = df_long.pivot(index=["Country", "Continent"], columns="Year", values="Population")
# df_wide

In [ ]:
df_wide.columns.name = None
df_wide = df_wide.reset_index()
df_wide

In [ ]:
pop_by_year = df_long.pivot_table(index="Year", columns="Continent", values="Population", aggfunc="sum")
pop_by_year

In [ ]:
pop_by_year.index = pd.to_datetime(pop_by_year.index, format="%Y").year
pop_by_year.index

In [ ]:
ax = pop_by_year.plot(kind="line", figsize=(16, 6), linewidth=3)
ax.set_title("Population by year")
ax.set_ylabel("Population (billion)")
ax.xaxis.set_ticks(pop_by_year.index)
plt.grid()
plt.show()

# Néhány algoritmus

Hogyan számolnánk ki ${}^3\sqrt{2}$-at, vagy $\sqrt{2}$-t 50 tizedesjegy pontossággal?

In [ ]:
import math

def print_answer(answer):
    answer = str(answer)
    print(f"{answer[0]}.", end="")
    for i in range(1, len(answer), 50):
        for j in range(i, i+50, 5):
            print(answer[j:j+5], end=" ")
        print("\n  ", end="")
    print()
    
n = 126
answer = math.isqrt(2*(10**(2*n)))
print_answer(answer)


In [ ]:
n = 100
answer = math.isqrt(2*(10**(2*n)))
print_answer(answer)


Mi a helyzet  ${}^3\sqrt{2}$-mal?

Az 
$$
f(x) = c-x^3
$$
függvény nullhelyét keressük. Pontosabban azt a legnagyobb $x$ egészet, amire $f(x)\geq 0$, ez a $^{3}\sqrt{c}$ egészrésze. 

Ha $c=2*10^{3*n}$, ekkor a megoldás  $[^{3}\sqrt{2}*10^n]$. Azaz az első $n$ tizedesjegy pontos lesz.

## Nullhely keresés

Ötlet, induljunk ki egy jó tippből és közelítsük a függvényünket lineáris függvénnyel.

$$
    f(x) \approx f(x_n)+(x-x_n) f'(x_n)
$$

és 

$$
    f(x_n) + (x-x_n) f'(x_n)=0 \quad\iff\quad  x = x_n - \frac{f(x_n)}{f'(x_n)}
$$

Rekurzió:
$$
x_{n+1} = x_n-\frac{f(x_n)}{f'(x_n)}
$$
Ha $f$ ,,szép'',  $f(t)=0$ és $x_n$ közel van $t$-hez, akkor

$$
\begin{aligned}
    x_{n+1} -t & = x_n-t -\frac{f(x_n)-f(t)}{f'(x_n)} \\
    & = (x_n-t)\left(1-\int_0^1 \frac{f'(t+u(x_n-t))}{f'(x_n)} du\right) \\
    & = (x_n-t)\int_0^1 \frac{f'(x_n) - f'(t+u(x_n-t))}{f'(x_n)} du
\end{aligned}
$$

és ha $x_n, x_{n+1}, t \in I$

$$
    |x_{n+1} -t|\leq \frac12|x_n-t|^2 \cdot \frac{\max_{u\in I}|f''(u)|}{\min_{u\in I} |f'(u)|}
$$

pl.

$f(x) = x^2-2$, $f'(x)=2x$,
  $f''(x)=2$ és ha $x_n, x_{n+1}, t \in [1,2]$, akkor $|x_{n+1}-t|\leq\frac12|x_n-t|^2$.

$f(x) = x^3-2$, $f'(x)=3x^2$, $f''(x)=6x$ és  $x_n, x_{n+1}, t \in [1,2]$, akkor $|x_{n+1}-t| \leq |x_n-t|^2$.


A rekurzió alakja $f(x) = x^3-c$-re.
$$
    x_{n+1} = x_n-\frac{f(x_n)}{f'(x_n)} = \frac{3x_n^3-(x_n^3-c)}{3 x_n^2}= \frac{2x_n}3 + \frac{c}{3x_n^2} 
$$

In [ ]:
import sys 
sys.set_int_max_str_digits(50000)

In [ ]:
def cube_root2(digits=1):
    x = 1
    c = 2
    delta = 1
    while digits > 0:
        delta = min(delta, digits)
        c *= 10**(3*delta)
        x *= 10**delta
        while True:
            x0 = x+1
            x = (2*x0+c//(x0**2))//3
            if x0 == x+1:
                break
            # print(delta)
        digits -= delta
        delta <<=  1

    return x


In [ ]:
n = 85

a = cube_root2(n)

a_cube = a**3

k = len(str(a_cube))-1

a_cube <= 2*10**(k) < (a+1)**3


In [ ]:
print_answer(a)

In [ ]:
from tqdm.auto import tqdm
import time
import sys 
sys.set_int_max_str_digits(50000)

timing = []
n_repeat = 10
for e in tqdm(range(1, 25)):
    n = int(1.5 ** e)
    start = time.perf_counter_ns()
    for _ in range(n_repeat):
        a = cube_root2(n)
    end = time.perf_counter_ns()

    timing.append({"run_time": (end-start)/n_repeat, "digits": len(str(a))-1})


In [ ]:
df  = pd.DataFrame(timing)
df.plot(x="digits", y="run_time", logx=True, logy=True)
plt.show()

In [ ]:
df